In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch.utils.data import Sampler


import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy import stats
import itertools
from itertools import product

In [ ]:
#keep the random state constant

class RandomState(object):
    def __init__(self, random_state=None):
        self.random_state = random_state
    def next(self, n=1):
        assert type(n) == int, "Ensure n is an integer"
        if n == 1:
            self.random_state,\
                out_state = np.random.default_rng(
                    self.random_state
                    ).integers(0, 1e9, size=(2,))
        else:
            self.random_state,\
                *out_state = np.random.default_rng(
                    self.random_state
                    ).integers(0, 1e9, size=(n+1,))
        
        return out_state

random_state = RandomState(42)
torch.manual_seed(random_state.next())

In [ ]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

## Baseline

In [ ]:
# Function to assign source names to the dataset equally
def assign_sources_equally(dataset, sources=('A', 'B', 'C', 'D', 'E', 'F')):
    num_sources = len(sources)
    num_data = len(dataset)
    num_each = num_data // num_sources
    
    # Convert sources to a list if it's a tuple
    sources_list = list(sources)
    
    # Create a list of source labels, each repeated equally
    source_labels = sources_list * num_each + [sources_list[i] for i in range(num_data % num_sources)]
    
    # Shuffle the labels to randomize their order
    np.random.shuffle(source_labels)
    
    return source_labels

# Assign sources to training and test data
training_sources = assign_sources_equally(training_data)
test_sources = assign_sources_equally(test_data)

# Example of how you can use the assigned sources
print("First 10 source labels for training data:", training_sources[:10])

In [ ]:
class CustomMNIST(Dataset):
    def __init__(self, mnist_dataset, source_labels, transform=None):
        """
        Custom dataset that applies a transform conditionally based on source labels.
        
        Args:
        mnist_dataset (Dataset): The original MNIST dataset.
        source_labels (list): List of source labels for each image in mnist_dataset.
        transform (callable, optional): A function/transform to apply to the images.
        """
        self.mnist_dataset = mnist_dataset
        self.source_labels = source_labels
        self.transform = transform

    def __len__(self):
        return len(self.mnist_dataset)

    def __getitem__(self, idx):
        image, label = self.mnist_dataset[idx]
        source_label = self.source_labels[idx]

        if source_label == 'A':
            # Rotate the image by 180 degrees
            image = transforms.functional.rotate(image, 180)
        elif source_label == 'B':
            # Randomly set 1/2 of the pixels to white (missing)
            c, h, w = image.shape
            num_pixels = h * w
            num_missing = num_pixels // 2
            mask = torch.randperm(num_pixels)[:num_missing]
            mask_h, mask_w = mask // w, mask % w
            image[:, mask_h, mask_w] = 1
        elif source_label == 'C':
            # Introduce noise
            noise = torch.randn_like(image) * 0.5  # Adjust noise level as needed
            image = image + noise
            image = torch.clamp(image, 0, 1)  # Ensure pixel values are still valid
        elif source_label == 'D':
            # Apply label permutation for source D
            label_permutation = {0: 9, 1: 8, 2: 7, 3: 6, 4: 5, 5: 4, 6: 3, 7: 2, 8: 1, 9: 0}
            label = label_permutation[label]
        elif source_label == 'E':
            #keep the image as is
            pass
        elif source_label == 'F':
            pass
        else:
            raise ValueError("Unknown source label provided: must be 'A', 'B', or 'C', or 'D', or 'E'")


        if self.transform:
            image = self.transform(image)

        image = torch.flatten(image)

        return image, label, source_label


# Create custom datasets
custom_train_dataset = CustomMNIST(training_data, training_sources, transform=None)
custom_test_dataset = CustomMNIST(test_data, test_sources, transform=None)


In [ ]:
# Custom model class with dropout
class TunedMLP(nn.Module):
    def __init__(self, input_size, output_size, dropout_rate):
        super(TunedMLP, self).__init__()
        self.linear1 = nn.Linear(input_size, 100)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.linear2 = nn.Linear(100, 100)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.linear3 = nn.Linear(100, output_size)
    
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.dropout1(x)
        x = F.relu(self.linear2(x))
        x = self.dropout2(x)
        y_pred = self.linear3(x)
        return y_pred

In [ ]:
# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        source_correct = {source: 0 for source in 'ABCDEF'}
        source_total = {source: 0 for source in 'ABCDEF'}

        for images, labels, source_labels in train_loader:
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

            for source, pred, label in zip(source_labels, predicted, labels):
                if pred == label:
                    source_correct[source] += 1
                source_total[source] += 1

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = (correct_predictions / total_predictions) * 100

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')
        for source in source_correct:
            if source_total[source] > 0:
                source_accuracy = (source_correct[source] / source_total[source]) * 100
                print(f'Accuracy for Source {source}: {source_accuracy:.2f}%')

In [ ]:
# Hyperparameter grid
hyperparameter_grid = {
    'lr': [0.001, 0.005, 0.01],
    'dropout_rate': [0, 0.2, 0.5]
}

# Cross-Validation for Hyperparameter Tuning
kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_accuracy = 0
best_model = None
best_hyperparams = None

# Create all possible combinations of hyperparameters
all_combinations = list(product(*hyperparameter_grid.values()))

In [ ]:
for comb in all_combinations:
    lr, dropout_rate = comb
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(custom_train_dataset)):
        print(f'Fold {fold + 1} for hyperparameters: {comb}')

        train_subset = Subset(custom_train_dataset, train_idx)
        val_subset = Subset(custom_train_dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=128, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=128, shuffle=False)

        model = TunedMLP(input_size=784, output_size=10, dropout_rate=dropout_rate)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()

        train_model(model, train_loader, criterion, optimizer, num_epochs=5)

        model.eval()
        val_correct_predictions = 0
        val_total_predictions = 0
        source_correct = {source: 0 for source in 'ABCDEF'}
        source_total = {source: 0 for source in 'ABCDEF'}

        with torch.no_grad():
            for images, labels, source_labels in val_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                val_correct_predictions += (predicted == labels).sum().item()
                val_total_predictions += labels.size(0)

                for source, pred, label in zip(source_labels, predicted, labels):
                    if pred == label:
                        source_correct[source] += 1
                    source_total[source] += 1

        val_accuracy = val_correct_predictions / val_total_predictions * 100.0
        fold_results.append(val_accuracy)

        print(f'Validation Accuracy for Fold {fold + 1}: {val_accuracy:.2f}%')
        for source in source_correct:
            if source_total[source] > 0:
                source_accuracy = (source_correct[source] / source_total[source]) * 100
                print(f'Validation Accuracy for Source {source}: {source_accuracy:.2f}%')

    mean_val_accuracy = np.mean(fold_results)
    std_val_accuracy = np.std(fold_results)
    
    print(f'Mean Validation Accuracy: {mean_val_accuracy:.2f}%')
    print(f'Standard Deviation of Validation Accuracy: {std_val_accuracy:.2f}%')

    if mean_val_accuracy > best_accuracy:
        best_accuracy = mean_val_accuracy
        best_hyperparams = {'lr': lr, 'dropout_rate': dropout_rate}

print(f"Best Hyperparameters: {best_hyperparams}")

In [ ]:
# Bootstrap accuracy function
def bootstrap_accuracy(train_dataset, test_loader, best_hyperparams, num_bootstrap=5, sample_percentage=0.8):
    accuracies = []
    source_accuracies = {source: [] for source in 'ABCDEF'}

    for _ in range(num_bootstrap):
        bootstrap_indices = np.random.choice(len(train_dataset), size=int(len(train_dataset) * sample_percentage), replace=True)
        bootstrap_subset = Subset(train_dataset, bootstrap_indices)
        bootstrap_loader = DataLoader(bootstrap_subset, batch_size=128, shuffle=True)

        model = TunedMLP(input_size=784, output_size=10, dropout_rate=best_hyperparams['dropout_rate'])
        optimizer = torch.optim.Adam(model.parameters(), lr=best_hyperparams['lr'])
        criterion = nn.CrossEntropyLoss()

        train_model(model, bootstrap_loader, criterion, optimizer, num_epochs=10)

        model.eval()
        correct_predictions = 0
        total_predictions = 0

        source_correct = {source: 0 for source in 'ABCDEF'}
        source_total = {source: 0 for source in 'ABCDEF'}

        with torch.no_grad():
            for images, labels, source_labels in test_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                correct_predictions += (predicted == labels).sum().item()
                total_predictions += labels.size(0)

                for source, pred, label in zip(source_labels, predicted, labels):
                    if pred == label:
                        source_correct[source] += 1
                    source_total[source] += 1

        overall_accuracy = correct_predictions / total_predictions * 100
        accuracies.append(overall_accuracy)

        for source in source_correct:
            if source_total[source] > 0:
                source_accuracy = (source_correct[source] / source_total[source]) * 100
                source_accuracies[source].append(source_accuracy)

    overall_mean = np.mean(accuracies)
    overall_std = np.std(accuracies)

    source_means = {source: np.mean(source_accuracies[source]) for source in source_accuracies}
    source_stds = {source: np.std(source_accuracies[source]) for source in source_accuracies}

    return overall_mean, overall_std, source_means, source_stds

# Evaluate best model on test set using bootstrap sampling
test_loader = DataLoader(custom_test_dataset, batch_size=128, shuffle=False)

print("Bootstrap evaluation on test set:")
test_mean, test_std, test_source_means, test_source_stds = bootstrap_accuracy(custom_train_dataset, test_loader, best_hyperparams)

print(f"Overall Test Accuracy: {test_mean:.2f}% ± {test_std:.2f}%")
for source in test_source_means:
    print(f"Test Accuracy for Source {source}: {test_source_means[source]:.2f}% ± {test_source_stds[source]:.2f}%")

# Source fully separate

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Model with separate layers for each source
class SourceMLP(nn.Module):
    def __init__(self, input_size, output_size, dropout_rate):
        super(SourceMLP, self).__init__()
        self.networks = nn.ModuleDict({
            source: nn.Sequential(
                nn.Linear(input_size, 100),
                nn.ReLU(),
                nn.Dropout(dropout_rate),
                nn.Linear(100, 100),
                nn.ReLU(),
                nn.Dropout(dropout_rate),
                nn.Linear(100, output_size)
            ) for source in 'ABCDEF'
        })

    def forward(self, x, source_labels):
        outputs = [self.networks[source](x[i].unsqueeze(0)) for i, source in enumerate(source_labels)]
        return torch.cat(outputs)

In [ ]:
# Train and validate function
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for images, labels, sources in train_loader:
            optimizer.zero_grad()
            outputs = model(images, sources)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        train_loss = total_loss / total
        train_acc = correct / total * 100

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        source_counts = {s: 0 for s in 'ABCDEF'}
        source_correct = {s: 0 for s in 'ABCDEF'}
        with torch.no_grad():
            for images, labels, sources in val_loader:
                outputs = model(images, sources)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * labels.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)
                for i, source in enumerate(sources):
                    source_counts[source] += 1
                    if predicted[i] == labels[i]:
                        source_correct[source] += 1

        val_loss /= val_total
        val_acc = val_correct / val_total * 100
        source_acc = {s: (source_correct[s] / source_counts[s] * 100) if source_counts[s] > 0 else 0 for s in 'ABCDEF'}

        print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        for s, acc in source_acc.items():
            print(f'Source {s}: Validation Accuracy: {acc:.2f}%')
    
    return val_acc, source_acc

In [ ]:
# Cross-validation and hyperparameter tuning
kf = KFold(n_splits=5, shuffle=True)
hyperparameter_grid = {'lr': [0.001, 0.005, 0.01], 'dropout_rate': [0, 0.2, 0.5]}
best_accuracy = 0
best_params = None
best_model = None

for lr, dropout_rate in itertools.product(*hyperparameter_grid.values()):
    fold_accuracies = []
    fold_source_accuracies = {source: [] for source in 'ABCDEF'}
    print(f'Testing parameters: lr={lr}, dropout={dropout_rate}')
    for fold, (train_idx, val_idx) in enumerate(kf.split(custom_train_dataset)):
        print(f'Starting Fold {fold+1}')
        train_subset = Subset(custom_train_dataset, train_idx)
        val_subset = Subset(custom_train_dataset, val_idx)
        train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)

        model = SourceMLP(784, 10, dropout_rate)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()

        accuracy, source_accuracies = train_and_validate(model, train_loader, val_loader, criterion, optimizer, 10)
        fold_accuracies.append(accuracy)
        for source in fold_source_accuracies:
            fold_source_accuracies[source].append(np.mean(source_accuracies[source]))

    mean_accuracy = np.mean(fold_accuracies)
    std_accuracy = np.std(fold_accuracies)
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_params = {'lr': lr, 'dropout_rate': dropout_rate}
        best_model = model

    print(f'Parameters: lr={lr}, dropout={dropout_rate}, Mean Accuracy: {mean_accuracy:.2f}%, Std Dev: {std_accuracy:.2f}%')
    for source in fold_source_accuracies:
        mean_source = np.mean(fold_source_accuracies[source])
        std_source = np.std(fold_source_accuracies[source])
        print(f'Source {source} - Mean: {mean_source:.2f}%, Std Dev: {std_source:.2f}%')

print(f'Best Model Parameters: {best_params}, with accuracy: {best_accuracy:.2f}%, Std Dev: {std_accuracy:.2f}%')


In [ ]:
# Bootstrap sampling and testing
bootstrap_accuracies = []
source_bootstrap_accuracies = {source: [] for source in 'ABCDEF'}
for i in range(5):
    indices = np.random.choice(len(custom_train_dataset), size=int(0.8 * len(custom_train_dataset)), replace=False)
    bootstrap_subset = Subset(custom_train_dataset, indices)
    bootstrap_loader = DataLoader(bootstrap_subset, batch_size=64, shuffle=True)
    test_loader = DataLoader(custom_test_dataset, batch_size=64, shuffle=False)

    # Initialize the model with the best hyperparameters
    model = SourceMLP(784, 10, best_params['dropout_rate'])
    optimizer = torch.optim.Adam(model.parameters(), lr=best_params['lr'])
    criterion = nn.CrossEntropyLoss()

    print(f'Bootstrap iteration {i+1}')
    test_accuracy, source_accuracies = train_and_validate(model, bootstrap_loader, test_loader, criterion, optimizer, 10)
    bootstrap_accuracies.append(test_accuracy)
    for source in source_accuracies:
        source_bootstrap_accuracies[source].append(np.mean(source_accuracies[source]))

mean_bootstrap = np.mean(bootstrap_accuracies)
std_bootstrap = np.std(bootstrap_accuracies)
print(f'Bootstrap results: Mean accuracy: {mean_bootstrap:.2f}%, Std Dev: {std_bootstrap:.2f}%')
for source in source_bootstrap_accuracies:
    mean_source = np.mean(source_bootstrap_accuracies[source])
    std_source = np.std(source_bootstrap_accuracies[source])
    print(f'Source {source} - Bootstrap Mean: {mean_source:.2f}%, Std Dev: {std_source:.2f}%')

# Source final layer separate

In [ ]:
class SourceMLP2(nn.Module):
    def __init__(self, input_size, output_size, dropout_rate):
        super(SourceMLP2, self).__init__()
        self.linear1 = nn.Linear(input_size, 100)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.linear2 = nn.Linear(100, 100)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.final_layers = nn.ModuleDict({
            'A': nn.Linear(100, output_size),
            'B': nn.Linear(100, output_size),
            'C': nn.Linear(100, output_size),
            'D': nn.Linear(100, output_size),
            'E': nn.Linear(100, output_size),
            'F': nn.Linear(100, output_size)
        })

    def forward(self, data, sources):
        x = F.relu(self.linear1(data))
        x = self.dropout1(x)
        x = F.relu(self.linear2(x))
        x = self.dropout2(x)
        outputs = [self.final_layers[source](x[i].unsqueeze(0)) for i, source in enumerate(sources)]
        return torch.cat(outputs)

In [ ]:
# Train and validate function
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for images, labels, sources in train_loader:
            optimizer.zero_grad()
            outputs = model(images, sources)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        train_loss = total_loss / total
        train_acc = correct / total * 100

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        source_counts = {s: 0 for s in 'ABCDEF'}
        source_correct = {s: 0 for s in 'ABCDEF'}
        with torch.no_grad():
            for images, labels, sources in val_loader:
                outputs = model(images, sources)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * labels.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()
                val_total += labels.size(0)
                for i, source in enumerate(sources):
                    source_counts[source] += 1
                    if predicted[i] == labels[i]:
                        source_correct[source] += 1

        val_loss /= val_total
        val_acc = val_correct / val_total * 100
        source_acc = {s: (source_correct[s] / source_counts[s] * 100) if source_counts[s] > 0 else 0 for s in 'ABCDEF'}

        print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        for s, acc in source_acc.items():
            print(f'Source {s}: Validation Accuracy: {acc:.2f}%')
    
    return val_acc, source_acc

In [ ]:
# Cross-validation and hyperparameter tuning
kf = KFold(n_splits=5, shuffle=True)
hyperparameter_grid = {'lr': [0.001, 0.005, 0.01], 'dropout_rate': [0, 0.2, 0.5]}
best_accuracy = 0
best_params = None
best_model = None

for lr, dropout_rate in itertools.product(*hyperparameter_grid.values()):
    fold_accuracies = []
    fold_source_accuracies = {source: [] for source in 'ABCDEF'}
    print(f'Testing parameters: lr={lr}, dropout={dropout_rate}')
    for fold, (train_idx, val_idx) in enumerate(kf.split(custom_train_dataset)):
        print(f'Starting Fold {fold+1}')
        train_subset = Subset(custom_train_dataset, train_idx)
        val_subset = Subset(custom_train_dataset, val_idx)
        train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)

        model = SourceMLP2(784, 10, dropout_rate)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()

        accuracy, source_accuracies = train_and_validate(model, train_loader, val_loader, criterion, optimizer, 10)
        fold_accuracies.append(accuracy)
        for source in fold_source_accuracies:
            fold_source_accuracies[source].append(np.mean(source_accuracies[source]))

    mean_accuracy = np.mean(fold_accuracies)
    std_accuracy = np.std(fold_accuracies)
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_params = {'lr': lr, 'dropout_rate': dropout_rate}
        best_model = model

    print(f'Parameters: lr={lr}, dropout={dropout_rate}, Mean Accuracy: {mean_accuracy:.2f}%, Std Dev: {std_accuracy:.2f}%')
    for source in fold_source_accuracies:
        mean_source = np.mean(fold_source_accuracies[source])
        std_source = np.std(fold_source_accuracies[source])
        print(f'Source {source} - Mean: {mean_source:.2f}%, Std Dev: {std_source:.2f}%')

print(f'Best Model Parameters: {best_params}, with accuracy: {best_accuracy:.2f}%, Std Dev: {std_accuracy:.2f}%')

# Bootstrap sampling and testing
bootstrap_accuracies = []
source_bootstrap_accuracies = {source: [] for source in 'ABCDEF'}
for i in range(5):
    indices = np.random.choice(len(custom_train_dataset), size=int(0.8 * len(custom_train_dataset)), replace=False)
    bootstrap_subset = Subset(custom_train_dataset, indices)
    bootstrap_loader = DataLoader(bootstrap_subset, batch_size=64, shuffle=True)
    test_loader = DataLoader(custom_test_dataset, batch_size=64, shuffle=False)

    print(f'Bootstrap iteration {i+1}')
    test_accuracy, source_accuracies = train_and_validate(best_model, bootstrap_loader, test_loader, criterion, optimizer, 10)
    bootstrap_accuracies.append(test_accuracy)
    for source in source_accuracies:
        source_bootstrap_accuracies[source].append(np.mean(source_accuracies[source]))

mean_bootstrap = np.mean(bootstrap_accuracies)
std_bootstrap = np.std(bootstrap_accuracies)
print(f'Bootstrap results: Mean accuracy: {mean_bootstrap:.2f}%, Std Dev: {std_bootstrap:.2f}%')
for source in source_bootstrap_accuracies:
    mean_source = np.mean(source_bootstrap_accuracies[source])
    std_source = np.std(source_bootstrap_accuracies[source])
    print(f'Source {source} - Bootstrap Mean: {mean_source:.2f}%, Std Dev: {std_source:.2f}%')